In [ ]:
year_start = 2014
year_end = 2018

In [ ]:
import requests

import pandas as pd

import holoviews as hv
from holoviews import opts, dim

hv.extension('bokeh','matplotlib')

In [ ]:
def get_visits_objects_years(site_id, year_start, year_end):

    endpoint = 'https://stats.data.gouv.fr/'
    
    params = {
        'idSite': site_id,
        'module': 'API',
        'method': 'Actions.getPageUrls',
        'format': 'json',
        'period': 'year',
        'date': '{0}-01-01,{1}-12-31'.format(year_start, year_end),
        'expanded': 1,
        'depth': 2,
        #'filter_pattern': '\/fr\/.*',
        'filter_limit': -1
    }

    r = requests.get(endpoint, params=params)
    print(r.url)
    # df = pd.DataFrame(r.json()).transpose()

    return r.json()

data = get_visits_objects_years(1, year_start, year_end)

len(data)

In [ ]:
select = [
    "datasets",
    "reuses",
    "topics",
    "organizations",
    "territories",
    "posts",
    "dashboard",
    "users"
]

In [ ]:
df = pd.DataFrame([ {"year" : y, "object": st["label"], "visits": st["nb_visits"] } for y in data for t in data[y] if t["label"] == "fr" for st in t["subtable"] if st["label"] in select ])

In [ ]:
bars = hv.Table(df, ["year", "object"], ["visits"]).to.bars(["year", "object"], 'visits', [])

bars.opts(width=800, height=400, stacked=True, tools=['hover'], legend_position="top")

In [ ]:
pd.options.display.float_format = '{}'.format
df.set_index("year").pivot(columns="object")

In [ ]:
pd.options.display.float_format = '{:.2%}'.format
df.set_index("year").pivot(columns="object").pct_change()